In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import requests
from io import StringIO
from time import sleep
import re
import time

from pathlib import Path
from typing import Iterable, Dict, Set

In [2]:
candidates_path = "alpha_helix_candidates.txt" # populated from HTML of website
table_schema = ["PDBID", "Protein Name", "Family", "UniprotID", "RES", "Text", "Organism"]

series_l = []
with open(candidates_path, "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith("<tr><td>") and line.endswith("</td></tr>"):
            line = line[8:-10]
        else:
            continue # poorly formatted line
        l = line.split("</td><td>")
        d = {table_schema[i] : l[i] for i in range(len(l))}
        series_l.append(d)

df_candidates = pd.DataFrame(series_l)
df_candidates

,PDBID,Protein Name,Family,UniprotID,RES,Text,Organism
0,5bz3,Na(+)/H(+) antiporter,Sodium-proton antiporter 1,Q5SIA2,2.30,CRYSTAL STRUCTURE OF SODIUM PROTON ANTIPORTER ...,Thermus thermophilus (strain HB8 / ATCC 27634 ...
1,4qtn,Nicotinamide riboside transporter PnuC,Nicotinamide mononucleotide transporter,D2ZZC1,2.80,Crystal structure of the Vitamin B3 transporte...,Neisseria mucosa ATCC 25996
2,5ezm,4-amino-4-deoxy-L-arabinose transferase or rel...,Aminoarabinose transferase family,Q1LDT6,2.70,Crystal Structure of ArnT from Cupriavidus met...,Cupriavidus metallidurans (strain ATCC 43123 /...
3,4od4,4-hydroxybenzoate octaprenyltransferase,UbiA prenyltransferases,Q9YBM8,3.30,Apo structure of a UbiA homolog from Aeropyrum...,Aeropyrum pernix (strain ATCC 700893 / DSM 118...
4,4p19,425aa long hypothetical proton glutamate sympo...,Proton glutamate symport protein,O59010,3.25,"Closed, apo inward-facing state of the glutama...",Pyrococcus horikoshii (strain ATCC 700860 / DS...
...,...,...,...,...,...,...,...
425,3vou,Voltage-gated sodium channel,Voltage-gated ion channels,A3SUL8,3.20,The crystal structure of NaK-NavSulP chimera c...,Sulfitobacter sp. NAS-14.1
426,4g7y,Voltage-sensor containing phosphatase,Voltage-sensor domain,Q4W8A1,2.80,Crystal structure of voltage sensing domain of...,Ciona intestinalis (Transparent sea squirt) (A...
427,5c78,WlaB protein,ABC transporter B family (ABCB),O86150,2.90,ATP-driven lipid-linked oligosaccharide flippa...,Campylobacter jejuni
428,3ddl,Xanthorhodopsin,Ion-translocating microbial rhodopsin,Q2S2F8,1.90,"Crystallographic Structure of Xanthorhodopsin,...",Salinibacter ruber


In [3]:
UNI_BASE = "https://rest.uniprot.org/uniprotkb/stream"
ACCESSION_RE = re.compile(r"^[A-NR-Z][0-9][A-Z0-9]{3}[0-9](?:-\d+)?$")  # supports isoforms (P12345-2)

def _normalize_uniprot_ids(series) -> list:
    """Split dirty cells, strip junk, validate accessions, dedupe while preserving order."""
    seen, clean = set(), []
    for raw in series.fillna("").astype(str):
        for tok in re.split(r"[^\w\-]+", raw):  # split on non-word/non-dash
            tok = tok.strip().upper().replace("_", "")
            if tok and ACCESSION_RE.match(tok) and tok not in seen:
                seen.add(tok)
                clean.append(tok)
    return clean

def _chunk(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

def _query_chunk(chunk, timeout=20):
    q = "accession:(" + " OR ".join(chunk) + ")"
    params = {"format": "tsv", "fields": "accession,length", "query": q}
    r = requests.get(UNI_BASE, params=params, timeout=timeout)
    if r.status_code == 429:
        time.sleep(1.0)
        r = requests.get(UNI_BASE, params=params, timeout=timeout)
    r.raise_for_status()
    return pd.read_csv(StringIO(r.text), sep="\t")

def fetch_uniprot_lengths(uniprot_ids, chunk_size=100, pause=0.2, timeout=20):
    """
    Batch-fetch sequence lengths for a list/Series of UniProt accessions.
    Returns DataFrame with columns ['Entry','Length'].
    Cleans/validates IDs and auto-recovers from 400 errors by shrinking chunks.
    """
    ids = _normalize_uniprot_ids(pd.Series(uniprot_ids))
    if not ids:
        return pd.DataFrame(columns=["Entry", "Length"])

    out = []
    try:
        for ch in _chunk(ids, chunk_size):
            df = _query_chunk(ch, timeout=timeout)
            if not df.empty:
                out.append(df[["Entry", "Length"]])
            if pause:
                time.sleep(pause)
    except requests.HTTPError as e:
        if e.response is not None and e.response.status_code == 400:
            # retry with smaller chunks; final fallback per-ID
            for ch in _chunk(ids, 20):
                try:
                    df = _query_chunk(ch, timeout=timeout)
                    if not df.empty:
                        out.append(df[["Entry", "Length"]])
                    if pause:
                        time.sleep(pause)
                except requests.HTTPError as e2:
                    if e2.response is not None and e2.response.status_code == 400:
                        for single in ch:
                            try:
                                df1 = _query_chunk([single], timeout=timeout)
                                if not df1.empty:
                                    out.append(df1[["Entry", "Length"]])
                            except requests.HTTPError:
                                pass  # skip truly bad IDs
                    else:
                        raise
        else:
            raise

    return pd.concat(out, ignore_index=True) if out else pd.DataFrame(columns=["Entry", "Length"])

def add_seq_length_from_uniprot(df, overwrite_res=False, new_col_name=None):
    """
    Merge UniProt sequence length into your DataFrame (by 'UniprotID').
    - overwrite_res=True: fill/overwrite df['RES'] from UniProt length.
    - else: create a new column ('SeqLen' or custom).
    """
    lens = fetch_uniprot_lengths(df["UniprotID"])
    merged = df.merge(lens, how="left", left_on="UniprotID", right_on="Entry").drop(columns=["Entry"])
    if overwrite_res:
        merged["RES"] = pd.to_numeric(merged.get("RES"), errors="coerce").fillna(merged["Length"])
        return merged.drop(columns=["Length"])
    else:
        col = new_col_name or "SeqLen"
        merged[col] = merged["Length"]
        return merged.drop(columns=["Length"])


In [4]:
# Fill RES in-place from UniProt:
df_candidates = add_seq_length_from_uniprot(df_candidates, overwrite_res=False, new_col_name="SeqLen").dropna(subset=["SeqLen"])
len_limit = 256
df_candidates = df_candidates[df_candidates["SeqLen"] < len_limit].reset_index(drop=True)
df_candidates 

,PDBID,Protein Name,Family,UniprotID,RES,Text,Organism,SeqLen
0,4wav,Bacteriorhodopsin-I,Microbial and algal rhodopsins,G0LFX8,2.80,Crystal Structure of Haloquadratum walsbyi bac...,Haloquadratum walsbyi (strain DSM 16854 / JCM ...,254.0
1,4fbz,Deltarhodopsin,Microbial and algal rhodopsins,I4DST7,2.70,Crystal structure of deltarhodopsin from Halot...,Haloterrigena thermotolerans,241.0
2,4bem,F-type ATP synthase subunit E,V-type and F-type ATPases,H6LFT2,2.10,Crystal structure of the F-type ATP synthase c...,Acetobacterium woodii (strain ATCC 29683 / DSM...,82.0
3,4tsy,Fragaceatoxin C,Anemone pore-forming cytolysin,B9W5G6,3.14,Crystal structure of FraC with lipids,Actinia fragacea (Strawberry anemone),179.0
4,5cbg,Ion transport 2 domain protein,Two-pore K+ channel,D5UM26,3.14,Calcium activated non-selective cation channel,Tsukamurella paurometabola (strain ATCC 8368 /...,123.0
5,2oar,Large-conductance mechanosensitive channel,Large conductance mechanosensitive ion channel...,A5U127,3.50,Mechanosensitive Channel of Large Conductance ...,Mycobacterium tuberculosis.,151.0
6,4wab,Leukotriene C4 synthase,MAPEG domain,B5MCC3,2.70,Crystal structure of mPGES1 solved by native-S...,Homo sapiens (Human),93.0
7,4rng,MtN3/saliva family,SemiSWEET bacterial sugar transporters,B5YGD6,2.40,Crystal structure of a bacterial homologue of ...,Thermodesulfovibrio yellowstonii (strain ATCC ...,88.0
8,5a43,Putative fluoride ion transporter CrcB,Fluoride exporter,B7LI20,2.58,Crystal structure of a dual topology fluoride ...,Escherichia coli O45:K1 (strain S88 / ExPEC),126.0
9,4hyj,Rhodopsin,Microbial and algal rhodopsins,B1YFV8,2.30,Crystal structure of Exiguobacterium sibiricum...,Exiguobacterium sibiricum (strain DSM 17290 / ...,252.0


In [10]:
def download_file(url, local_filename):
    """
    Downloads a file from a given URL and saves it locally.

    Args:
        url (str): The URL of the file to download.
        local_filename (str): The name to save the downloaded file as.
    """
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        with open(local_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"File '{local_filename}' downloaded successfully from '{url}'")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")

In [5]:
# now retrieve pdbs
good_pdbids = df_candidates["PDBID"]
print(good_pdbids)

0     4wav
1     4fbz
2     4bem
3     4tsy
4     5cbg
5     2oar
6     4wab
7     4rng
8     5a43
9     4hyj
10    5ax0
11    3am6
12    4pop
13    4qnc
14    4qnd
15    4xu4
Name: PDBID, dtype: object


In [12]:
folder = "alpha_helix_pdbs"
for id in good_pdbids:
    download_file(f"https://files.rcsb.org/download/{id}.pdb", f"{folder}/{id}.pdb")

File 'alpha_helix_pdbs/4wav.pdb' downloaded successfully from 'https://files.rcsb.org/download/4wav.pdb'
File 'alpha_helix_pdbs/4fbz.pdb' downloaded successfully from 'https://files.rcsb.org/download/4fbz.pdb'
File 'alpha_helix_pdbs/4bem.pdb' downloaded successfully from 'https://files.rcsb.org/download/4bem.pdb'
File 'alpha_helix_pdbs/4tsy.pdb' downloaded successfully from 'https://files.rcsb.org/download/4tsy.pdb'
File 'alpha_helix_pdbs/5cbg.pdb' downloaded successfully from 'https://files.rcsb.org/download/5cbg.pdb'
File 'alpha_helix_pdbs/2oar.pdb' downloaded successfully from 'https://files.rcsb.org/download/2oar.pdb'
File 'alpha_helix_pdbs/4wab.pdb' downloaded successfully from 'https://files.rcsb.org/download/4wab.pdb'
File 'alpha_helix_pdbs/4rng.pdb' downloaded successfully from 'https://files.rcsb.org/download/4rng.pdb'
File 'alpha_helix_pdbs/5a43.pdb' downloaded successfully from 'https://files.rcsb.org/download/5a43.pdb'
File 'alpha_helix_pdbs/4hyj.pdb' downloaded successfull

In [6]:
def parse_pdb(pdb_path):
    """
    Parse a PDB file line by line.
    Returns:
      atoms:  list of dicts (serial, name, resname, chain, resseq, x,y,z)
      helices: list of dicts (helix_id, start_res, start_chain, end_res, end_chain, length)
    """
    atoms, helices = [], []
    atom_lines = []
    helix_lines = []
    with open(pdb_path) as f:
        for line in f:
            record = line[0:6].strip()

            # ---- ATOM records ----
            if record == "ATOM":
                atoms.append({
                    "serial":   int(line[6:11]),
                    "name":     line[12:16].strip(),
                    "resname":  line[17:20].strip(),
                    "chain":    line[21].strip(),
                    "resseq":   int(line[22:26]),
                    "x":        float(line[30:38]),
                    "y":        float(line[38:46]),
                    "z":        float(line[46:54]),
                    "occ":      float(line[54:60] or 0),
                    "bfac":     float(line[60:66] or 0),
                })
                atom_lines.append(str(line))

            # ---- HELIX records ----
            elif record == "HELIX":
                helices.append({
                    "ser_num":    line[7:10].strip(),       # helix serial number
                    "helix_id":   line[11:14].strip(),      # helix identifier
                    "start_res":  line[15:18].strip(),      # initial residue name
                    "start_chain":line[19].strip(),         # chain identifier
                    "start_seq":  int(line[21:25]),         # start residue sequence number
                    "end_res":    line[27:30].strip(),      # ending residue name
                    "end_chain":  line[31].strip(),         # chain identifier
                    "end_seq":    int(line[33:37]),         # end residue sequence number
                    "helix_class":line[38:40].strip(),
                    "length":     int(line[71:76]),
                })
                helix_lines.append(str(line))

    return atom_lines, helices, helix_lines

In [14]:
# --- planning + header builder to keep total length under a cap (e.g., 200) ---

def _hdr_name(name:str) -> str:
    return f"REMARK 999 NAME   {name}\n"

def _hdr_pdb(pdbid:str) -> str:
    return f"REMARK 999 PDB    {pdbid}\n"

def _hdr_scaffold(min_len:int, max_len:int) -> str:
    # cols 1–16 label; scaffold min at col 20, max at 24
    return f"{'REMARK 999 INPUT':<16}   {min_len:>4}{max_len:>4}\n"

def _hdr_motif(chain:str, start:int, end:int, group:str='A') -> str:
    # cols 1–16 label; chain col 19; start 20–23; end 24–27; group at col 29
    return f"{'REMARK 999 INPUT':<16}  {chain:>1}{start:>4}{end:>4} {group}\n"

def _hdr_min_total(n:int) -> str:
    return f"{'REMARK 999 MINIMUM TOTAL LENGTH':<31}{'':<6}{n:<3}\n"

def _hdr_max_total(n:int) -> str:
    return f"{'REMARK 999 MAXIMUM TOTAL LENGTH':<31}{'':<6}{n:<3}\n"

def build_motif_scaffold_header(name, pdbid, motifs, smin, smax,
                                include_leading=False, include_trailing=False):
    assert motifs, "No motifs selected"
    motif_lengths = [end - start + 1 for (_, start, end, _) in motifs]
    n_scaff = max(0, len(motifs) - 1)
    if include_leading:  n_scaff += 1
    if include_trailing: n_scaff += 1
    min_total = sum(motif_lengths) + n_scaff * smin
    max_total = sum(motif_lengths) + n_scaff * smax

    lines = []
    lines.append(_hdr_name(name))
    lines.append(_hdr_pdb(pdbid))
    if include_leading:
        lines.append(_hdr_scaffold(smin, smax))
    for i, (ch, st, en, grp) in enumerate(motifs):
        lines.append(_hdr_motif(ch, st, en, grp))
        if i < len(motifs) - 1:
            lines.append(_hdr_scaffold(smin, smax))
    if include_trailing:
        lines.append(_hdr_scaffold(smin, smax))
    lines.append(_hdr_min_total(min_total))
    lines.append(_hdr_max_total(max_total))
    return "".join(lines), min_total, max_total

def plan_motifs_under_cap(helix_dicts, chain="A", cap_max=200, base_smin=4, base_smax=79):
    """
    Choose a small subset of chain-specific helices and a scaffold range so that MAX_TOTAL ≤ cap_max.
    Strategy: prefer shortest helices; try 3→2→1 motifs; if needed, tighten smax (not smin) to fit.
    Returns (motifs, smin, smax, min_total, max_total).
    """
    helices_c = [h for h in helix_dicts if h["start_chain"] == chain]
    if not helices_c:
        # fall back to global shortest helix
        h = min(helix_dicts, key=lambda x: (x["end_seq"] - x["start_seq"] + 1))
        helices_c = [h]
        chain = h["start_chain"]

    # sort by helix length ascending
    helices_sorted = sorted(helices_c, key=lambda h: (h["end_seq"] - h["start_seq"] + 1))
    # try subsets of size 3, then 2, then 1
    for k in (3, 2, 1):
        if len(helices_sorted) < k:
            continue
        cand = helices_sorted[:k]
        motifs = [(chain, h["start_seq"], h["end_seq"], "A") for h in cand]
        motif_sum = sum(h["end_seq"] - h["start_seq"] + 1 for h in cand)
        n_scaff = max(0, k - 1)  # scaffolds BETWEEN motifs only

        # start with base scaffold range
        smin, smax = base_smin, base_smax
        min_total = motif_sum + n_scaff * smin
        max_total = motif_sum + n_scaff * smax

        if min_total > cap_max:
            # even the minimum can't fit; try fewer motifs
            continue

        if max_total <= cap_max:
            return motifs, smin, smax, min_total, max_total

        # tighten smax just enough to fit
        if n_scaff > 0:
            needed_smax = (cap_max - motif_sum) // n_scaff
            smax_adj = max(smin, min(smax, needed_smax))
            max_total_adj = motif_sum + n_scaff * smax_adj
            if max_total_adj <= cap_max:
                return motifs, smin, smax_adj, min_total, max_total_adj

        # else try fewer motifs
    # If nothing fits (very long helices), fall back to the single shortest helix, no scaffold
    h = helices_sorted[0]
    motifs = [(chain, h["start_seq"], h["end_seq"], "A")]
    smin = smax = base_smin
    min_total = max_total = (h["end_seq"] - h["start_seq"] + 1)
    return motifs, smin, smax, min_total, max_total


In [17]:
# now retrieve residue numbers and atomic coords
folder = "alpha_helix_pdbs"
for id in good_pdbids:
    fname = f"{folder}/{id}.pdb"
    atoms, helices, helix_lines = parse_pdb(fname)
    out = []
    for i in range(len(helices)):
        row = {
            "helix_number" : helices[i]["helix_id"],
            "start_res_number": helices[i]["start_seq"],
            "end_res_number": helices[i]["end_seq"],
            "start_res_code": helices[i]["start_res"],
            "end_res_code": helices[i]["end_res"],
            "chain_identifier": helices[i]["end_chain"],
            "helix_class" : helices[i]["helix_class"],
            "length" : helices[i]["length"]
            }
        out.append(row)
    df_temp = pd.DataFrame(out)
    df_temp.to_csv(f"alpha_helix_residue_numbers/{id}.csv", index=False)
    
    # now process atomic coords
    atom_file_name = f"alpha_helix_atomic_coords/{id}_atoms.pdb"
    with open(atom_file_name, 'w') as f:
        for line in atoms:
            f.write(line)
    # now process helix coords/lines
    helix_file_name = f"alpha_helix_pdbs_helix_only/{id}_helix.pdb"
    with open(helix_file_name, 'w') as f:
        for line in helix_lines:
            f.write(line)
    # write combo with header 
    helix_file_name = f"alpha_helix_templates/{id}_helix.pdb"
    with open(helix_file_name, 'w') as f:
        # write header
        # write combo with header 
        helix_file_name = f"alpha_helix_templates/{id}_helix.pdb"
        with open(helix_file_name, 'w') as f:
            # plan motifs + scaffolds to keep max ≤ 200
            motifs, smin, smax, min_tot, max_tot = plan_motifs_under_cap(
                helices, chain="A", cap_max=200, base_smin=4, base_smax=79
            )

            header, min_total, max_total = build_motif_scaffold_header(
                name=f"guided_{id}",
                pdbid=id,
                motifs=motifs,
                smin=smin,
                smax=smax,
                include_leading=False,
                include_trailing=False,
            )
            # optional: quick sanity log
            print(id, motifs, (smin, smax), min_total, max_total)

            # write header then HELIX+ATOM
            f.write(header + "\n")
            for line in helix_lines:
                f.write(line)
            for line in atoms:
                f.write(line)

4wav [('A', 192, 200, 'A'), ('A', 172, 191, 'A'), ('A', 16, 38, 'A')] (4, 74) 60 200
4fbz [('A', 232, 234, 'A'), ('A', 192, 195, 'A'), ('A', 226, 231, 'A')] (4, 79) 21 171
4bem [('A', 2, 19, 'A'), ('A', 22, 43, 'A'), ('A', 46, 78, 'A')] (4, 63) 81 199
4tsy [('A', 129, 139, 'A'), ('A', 5, 29, 'A')] (4, 79) 40 115
5cbg [('A', 6, 12, 'A'), ('A', 49, 60, 'A'), ('A', 26, 45, 'A')] (4, 79) 47 197
2oar [('A', 1, 11, 'A'), ('A', 90, 100, 'A'), ('A', 105, 124, 'A')] (4, 79) 50 200
4wab [('A', 54, 59, 'A'), ('A', 46, 53, 'A'), ('A', 154, 161, 'A')] (4, 79) 30 180
4rng [('A', 23, 34, 'A'), ('A', 6, 23, 'A'), ('A', 40, 61, 'A')] (4, 74) 60 200
5a43 [('A', 28, 30, 'A'), ('A', 72, 78, 'A'), ('A', 62, 70, 'A')] (4, 79) 27 177
4hyj [('A', 33, 36, 'A'), ('A', 70, 74, 'A'), ('A', 95, 102, 'A')] (4, 79) 25 175
5ax0 [('A', 67, 69, 'A'), ('A', 167, 169, 'A'), ('A', 162, 165, 'A')] (4, 79) 18 168
3am6 [('A', 32, 34, 'A'), ('A', 179, 188, 'A'), ('A', 195, 211, 'A')] (4, 79) 38 188
4pop [('A', 38, 40, 'A'), (

In [ ]:
# folder = "alpha_helix_pdbs"
# for id in good_pdbids:
#     fname = f"{folder}/{id}.pdb"
#     atoms, helices, helix_lines = parse_pdb(fname)  # helices is defined here!

#     # --- residue CSV export ---
#     out = []
#     for h in helices:
#         row = {
#             "helix_number": h["helix_id"],
#             "start_res_number": h["start_seq"],
#             "end_res_number": h["end_seq"],
#             "start_res_code": h["start_res"],
#             "end_res_code": h["end_res"],
#             "chain_identifier": h["end_chain"],
#             "helix_class": h["helix_class"],
#             "length": h["length"],
#         }
#         out.append(row)
#     df_temp = pd.DataFrame(out)
#     df_temp.to_csv(f"alpha_helix_residue_numbers/{id}.csv", index=False)

#     # --- ATOM-only PDB ---
#     atom_file_name = f"alpha_helix_atomic_coords/{id}_atoms.pdb"
#     with open(atom_file_name, "w") as f:
#         for line in atoms:
#             f.write(line)

#     # --- HELIX-only PDB ---
#     helix_file_name = f"alpha_helix_pdbs_helix_only/{id}_helix.pdb"
#     with open(helix_file_name, "w") as f:
#         for line in helix_lines:
#             f.write(line)

#     # --- Template with REMARK 999 header + HELIX + ATOM ---
#     helix_file_name = f"alpha_helix_templates/{id}_helix.pdb"
#     with open(helix_file_name, "w") as f:
#         # select motifs (e.g., three short chain A helices)
#         helices_A = [h for h in helices if h["start_chain"] == "A"]

#         preferred = {(1, 11), (90, 100), (105, 124)}
#         motifs = [
#             ("A", h["start_seq"], h["end_seq"], "A")
#             for h in helices_A
#             if (h["start_seq"], h["end_seq"]) in preferred
#         ]

#         if len(motifs) < 3 and helices_A:
#             remaining = [h for h in helices_A if (h["start_seq"], h["end_seq"]) not in preferred]
#             remaining = sorted(remaining, key=lambda x: (x["end_seq"] - x["start_seq"] + 1))
#             for h in remaining[: 3 - len(motifs)]:
#                 motifs.append(("A", h["start_seq"], h["end_seq"], "A"))

#         if not motifs and helices:
#             h = min(helices, key=lambda x: (x["end_seq"] - x["start_seq"] + 1))
#             motifs = [(h["start_chain"], h["start_seq"], h["end_seq"], "A")]

#         header = build_motif_scaffold_header(
#             name=f"helices_small_total_{id}",
#             pdbid=id,
#             motifs=motifs,
#             scaffold_range=(4, 79),
#             include_leading_scaffold=False,
#             include_trailing_scaffold=False,
#         )

#         # write header, then HELIX lines, then ATOM lines
#         f.write(header + "\n")
#         for line in helix_lines:
#             f.write(line)
#         for line in atoms:
#             f.write(line)
